## Data Preprocessing

In [1]:
# Library imports
import pandas as pd
import numpy as np
from pulp import *

In [9]:
# Importing and checking out the dataset
df = pd.read_csv("servings.csv")
df.head()
df.columns

Index(['Day', 'Group', 'Food Name', 'Amount', 'Energy (kcal)', 'Alcohol (g)',
       'Caffeine (mg)', 'Water (g)', 'B1 (Thiamine) (mg)',
       'B2 (Riboflavin) (mg)', 'B3 (Niacin) (mg)',
       'B5 (Pantothenic Acid) (mg)', 'B6 (Pyridoxine) (mg)',
       'B12 (Cobalamin) (µg)', 'Folate (µg)', 'Vitamin A (µg)',
       'Vitamin C (mg)', 'Vitamin D (IU)', 'Vitamin E (mg)', 'Vitamin K (µg)',
       'Calcium (mg)', 'Copper (mg)', 'Iron (mg)', 'Magnesium (mg)',
       'Manganese (mg)', 'Phosphorus (mg)', 'Potassium (mg)', 'Selenium (µg)',
       'Sodium (mg)', 'Zinc (mg)', 'Carbs (g)', 'Fiber (g)', 'Starch (g)',
       'Sugars (g)', 'Added Sugars (g)', 'Net Carbs (g)', 'Fat (g)',
       'Cholesterol (mg)', 'Monounsaturated (g)', 'Polyunsaturated (g)',
       'Saturated (g)', 'Trans-Fats (g)', 'Omega-3 (g)', 'Omega-6 (g)',
       'Cystine (g)', 'Histidine (g)', 'Isoleucine (g)', 'Leucine (g)',
       'Lysine (g)', 'Methionine (g)', 'Phenylalanine (g)', 'Protein (g)',
       'Threonine (g)', 

In [3]:
# Group by 'Food Name' and collect unique 'Amount' for each group
grouped_df = df.groupby('Food Name')['Amount'].unique().reset_index()

# Expand the DataFrame so each unique 'Food Name' and 'Amount' is on a separate row
expanded_df = grouped_df.explode('Amount')

# Export the DataFrame to a CSV file
expanded_df.to_csv('grouped_food_names_amounts.csv')
expanded_df

,Food Name,Amount
0,"7-Eleven, Fresh, Breakfast Burger, Bacon, Egg ...",1.00 pack
1,"Apple, Fresh, With Skin","1.00 medium - 3"" diameter"
1,"Apple, Fresh, With Skin","0.25 large - 3 1/4"" diameter"
1,"Apple, Fresh, With Skin","0.75 large - 3 1/4"" diameter"
1,"Apple, Fresh, With Skin","1.00 large - 3 1/4"" diameter"
...,...,...
79,"Turo-turo Gourmet, Sisig Fully Cooked Stir Fri...",2.00 serving
80,"White Rice, Steamed",1.00 cup
80,"White Rice, Steamed",2.00 cup
80,"White Rice, Steamed",3.00 cup


In [4]:
df_cost = pd.read_csv("cost.csv").dropna()
df_cost.head()

,Food Name,Amount,Price
0,"7-Eleven, Fresh, Breakfast Burger, Bacon, Egg ...",1.00 pack,125.0
1,"Apple, Fresh, With Skin","1.00 medium - 3"" diameter",40.0
2,"Apple, Fresh, With Skin","0.25 large - 3 1/4"" diameter",20.0
3,"Apple, Fresh, With Skin","0.75 large - 3 1/4"" diameter",32.5
4,"Apple, Fresh, With Skin","1.00 large - 3 1/4"" diameter",47.5


In [ ]:
merged_df = pd.merge(df, df_cost, on=['Food Name', 'Amount'], how='inner')

specified_columns = ['Food Name', 'Amount', 'Energy (kcal)', 'Fiber (g)', 'Protein (g)', 'Potassium (mg)', 'Price']
final_df = merged_df[specified_columns].drop_duplicates()
final_df.fillna(0, inplace=True)
final_df.head()

,Food Name,Amount,Energy (kcal),Fiber (g),Protein (g),Price
0,"Eggs, Cooked",3.00 large,232.50,0.00,18.87,30.0
1,Argentina Corned Beef,1.00 can,180.00,2.00,12.00,35.0
2,"Quaker, Instant Oatmeal, Chocolate",32.00 g,129.94,2.72,3.88,30.0
3,"Banana, Fresh","1.00 medium - 7"" to 7 7/8"" long",105.02,3.07,1.29,12.5
4,C-Lium Fibre,1.00 Pack,30.00,5.00,0.00,23.0


In [ ]:
# Concatenate Amount into Food Name
final_df['Food Name'] = final_df['Food Name'] + ' ' + final_df['Amount'].astype(str)
food_names = final_df['Food Name'].tolist()

# Create dictionaries for 'Energy', 'Fiber', 'Protein', and 'Price'
energy_dict = final_df.set_index('Food Name')['Energy (kcal)'].to_dict()
fiber_dict = final_df.set_index('Food Name')['Fiber (g)'].to_dict()
fiber_dict['Gardenia, High Fiber Wheat Raisin Loaf 1.00 Slice'] = 3
fiber_dict['Gardenia, High Fiber Wheat Raisin Loaf 2.00 Slice'] = 6
protein_dict = final_df.set_index('Food Name')['Protein (g)'].to_dict()
potassium_dict = final_df.set_index('Food Name')['Potassium (mg)'].to_dict()
price_dict = final_df.set_index('Food Name')['Price'].to_dict()

# Display the results
print("Food Names Array:", food_names)
print("Energy Dictionary:", energy_dict)
print("Fiber Dictionary:", fiber_dict)
print("Protein Dictionary:", protein_dict)
print("Potassium Dictionary:", potassium_dict)
print("Price Dictionary:", price_dict)

Food Names Array: ['Eggs, Cooked 3.00 large', 'Argentina Corned Beef 1.00 can', 'Quaker, Instant Oatmeal, Chocolate 32.00 g', 'Banana, Fresh 1.00 medium - 7" to 7 7/8" long', 'C-Lium Fibre 1.00 Pack', 'Gardenia, High Fiber Wheat Raisin Loaf 2.00 Slice', 'Peanut Butter, Regular, Salted 1.00 tbsp', 'Hosen Baked Beans 1.00 can', 'Gardenia, High Fiber Wheat Raisin Loaf 1.00 Slice', 'Chicken Nuggets or Sticks, Store Bought 3.00 nugget', 'Pancake, Plain or Buttermilk, Homemade 3.00 each - 4" diameter', 'Collagen Vit C Zinc 1.00  capsule', 'Beef Steak, Sirloin, No Visible Fat Eaten 100.00 g', 'White Rice, Steamed 1.00 cup', 'Quaker, Oaties Mini Oat Cookies, Honey Nuts 1.00 Package', 'Smart C 1.00 x 350.0 ml', 'Bacon, Pork 8.00 slice - 6" long', 'Chicken with Gravy 1.00 cup', 'Nescafe, Instant Coffee  1.00 tsp', 'Century Tuna, Flakes in Oil 2.00 serving', 'Chicken patty, frozen, cooked 4.00 patty', 'Hamburger Bun, White 2.00 medium - 3 1/2" diameter', 'Kalbe, Fitbar, Chocolate 1.00 bar', 'Sard

## Optimization

In [ ]:
# Set variables
min_protein = 120
min_fiber = 40
max_energy = 1500
min_potassium = 3400

# Just read the case study at https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html. They explain it way better than I ever could.
prob = LpProblem("Meal Optimization", LpMinimize)
food_vars = LpVariable.dicts("Food", food_names, 0)
prob += (
    lpSum([price_dict[i] * food_vars[i] for i in food_names]),
    "Total Cost of Food daily",
)
prob += (
    lpSum([energy_dict[i] * food_vars[i] for i in food_names]) <= max_energy,
    "EnergyRequirement",
)
prob += (
    lpSum([fiber_dict[i] * food_vars[i] for i in food_names]) >= min_fiber,
    "FiberRequirement",
)
prob += (
    lpSum([protein_dict[i] * food_vars[i] for i in food_names]) >= min_protein,
    "ProteinRequirement",
)
prob += (
    lpSum([potassium_dict[i] * food_vars[i] for i in food_names]) >= min_potassium,
    "PotassiumRequirement",
)
prob.writeLP("MealOptimization.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
print("Total Cost of Food per day = ", value(prob.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/will/Projects/dev_blog_notebooks/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/a5967060de714a6fbeed65470c33ef7d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/a5967060de714a6fbeed65470c33ef7d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 427 RHS
At line 431 BOUNDS
At line 432 ENDATA
Problem MODEL has 3 rows, 118 columns and 300 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 112 (-6) columns and 295 (-5) elements
0  Obj 0 Primal inf 1.8241194 (2)
2  Obj 138.3916
Optimal - objective value 138.3916
After Postsolve, objective 138.3916, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 138.3915953 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU se

/home/will/Projects/dev_blog_notebooks/.venv/lib/python3.12/site-packages/pulp/pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
